In [1]:
from transformers import BertModel, BertTokenizer, BertConfig, BertForMaskedLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import random
import logging
import math

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


In [2]:
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
random.seed(0)

In [3]:
# Load pre-trained model tokenizer (vocabulary) and model
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForMaskedLM.from_pretrained('bert-base-cased')
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [4]:
def tokenize_input(text):
    tokenized_text = tokenizer.tokenize(text)
    
    sentence1 = tokenized_text[:tokenized_text.index(".")+1]
    sentence2 = tokenized_text[tokenized_text.index(".")+1:]
    
    sentence1 = ["[CLS]"] + sentence1
    sentence2 = ["[SEP]"] + sentence2 + ["[SEP]"]
    
    segments_ids = [0]*len(sentence1) + [1]*len(sentence2)
    new_tokenized_text = sentence1 + sentence2
    
    #print(len(sentence1))
    
    return new_tokenized_text, segments_ids

In [8]:
def mask_tokens(tokenized_text, mask_token_ids):
    for ids in mask_token_ids:
        #print(tokenized_text[id])
        tokenized_text[ids] = '[MASK]'
                
    # Convert tokenized text to their index
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    
    return tokenized_text, indexed_tokens

In [9]:
def produce_new_sentence(masked_tokenize, mask_token_ids, predictions):
    predicted = []

    for i in mask_token_ids:
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

        predicted.append(predicted_token)
    
    print(mask_token_ids)
    print(predicted)

    for word in predicted:
        mask_index = masked_tokenize.index("[MASK]")
        masked_tokenize[mask_index] = word

    final = ' '.join(masked_tokenize).replace(" ##", '').replace("[CLS] ", "").replace("[SEP] ", "").replace("[SEP]", "")

    print(final)
    
    return final

In [10]:
text = """My countrymen and women of the radio audience Of the untold values of the radio, one is the great intimacy it has brought among our people. 
The greatest strength of the radio is the intimacy of its listeners."""

cycles = 40

#number_masked = int((len(tokenized_text) - period_loc+1) / 5)
number_masked = 1

tokenized_text, segments_ids = tokenize_input(text)

for c in range(cycles):
    period_loc = tokenized_text.index(".")
    
    mask_token_ids = sorted(random.sample(range(period_loc+2, len(tokenized_text)-1), number_masked))

    masked_tokenize, indexed_tokens = mask_tokens(tokenized_text, mask_token_ids)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    tokens_tensor = tokens_tensor.to('cuda')
    segments_tensors = segments_tensors.to('cuda')
    model.to('cuda')

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor, token_type_ids=segments_tensors)
        predictions = outputs[0]

    tokenized_text = produce_new_sentence(masked_tokenize, mask_token_ids, predictions)
    
    

[32]
['The']
My countrymen and women of the radio audience Of the untold values of the radio , one is the great intimacy it has brought among our people . The greatest strength of the radio is the intimacy of its listeners . 


TypeError: 'str' object does not support item assignment

In [35]:
# Round 2: Potentially Growing


def tokenize_input2(text):
    tokenized_text = tokenizer.tokenize(text)
    
    sentence1 = tokenized_text[:tokenized_text.index(".")+1]
    sentence2 = tokenized_text[tokenized_text.index(".")+1:]
    
    sentence1 = ["[CLS]"] + sentence1
    sentence2 = ["[SEP]"] + sentence2 + ["[SEP]"]
    
    new_tokenized_text = sentence1 + sentence2
        
    return new_tokenized_text, sentence1, sentence2


def mask_tokens(tokenized_text, mask_token_ids):
    for id in mask_token_ids:
        #print(f"{id} ie. {tokenized_text[id]} is masked")
        tokenized_text[id] = '[MASK]'
    
    #Convert tokenized text to their index
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    
    return tokenized_text, indexed_tokens


def predict_token(masked_tokenize, mask_token_ids, predictions):
    predicted = []

    for i in mask_token_ids:
        predicted_index = torch.argmax(predictions[0, i]).item()
        predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

        predicted.append(predicted_token)

    for word in predicted:
        mask_index = masked_tokenize.index("[MASK]")
        
        #print(f"Masked index: {mask_index} became {word}")
        masked_tokenize[mask_index] = word
        
        if word == "." and masked_tokenize[mask_index + 1] != "[SEP]":
            masked_tokenize.insert(mask_index + 1,"[SEP]")          
    
    return masked_tokenize

def run(text, cycles, mask_type, mask_value):
    
    tokenized_text, sentence1, sentence2 = tokenize_input2(text)
    period_loc = tokenized_text.index(".")
    segments_ids = [0]*len(sentence1) + [1]*len(sentence2)
    
    masked_tokenize_prior_length = 99999
    tokenized_text_length = len(tokenized_text)
    
    for c in range(cycles):
        random.seed(c)
        
        if mask_type == 'frac':
            number_masked = int((tokenized_text_length - period_loc+1) * mask_value)
        elif mask_type == 'val':
            number_masked = mask_value
            
        
        mask_token_ids = sorted(random.sample(range(period_loc+2, tokenized_text_length-1), number_masked))
        masked_tokenize, indexed_tokens = mask_tokens(tokenized_text, mask_token_ids)

        if len(tokenized_text) > tokenized_text_length:
            difference = len(tokenized_text) - tokenized_text_length
            segments_ids = segments_ids + [1]*difference
            tokenized_text_length += difference

        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])

        tokens_tensor = tokens_tensor.to('cuda')
        segments_tensors = segments_tensors.to('cuda')
        model.to('cuda')

        # Predict all tokens
        with torch.no_grad():
            outputs = model(tokens_tensor, token_type_ids=segments_tensors)
            predictions = outputs[0]

        tokenized_text = predict_token(masked_tokenize, mask_token_ids, predictions)

        #print(tokenized_text)
        #print("==============================")
    
    tokenized_text.remove("[CLS]")
    tokenized_text.remove("[SEP]")
    tokenized_text.remove("[SEP]")
    
    result = ' '.join(tokenized_text)
    
    return result

In [12]:
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')

def score(sentence):
    tokenize_input = gpt_tokenizer.tokenize(sentence)
    tensor_input = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_input)])
    
    loss=gpt_model(tensor_input, labels=tensor_input)[0].item()
    
    return math.exp(loss)

In [13]:
text = """All of which comprises a very substantial reserve in the country at the present time . The Reserve Bank of India has also made a large amount of deposits , which is a good thing ."""

In [14]:
score(text)

2.820022077396162

In [15]:
cycles = 100
number_masked = 2
result = run(text, cycles, 'frac', 0.2)
print(result)
score(result)

All of which comprises a very substantial reserve in the country at the present time . Although the government is not directly directly involved , in the present case , this is the the case .


2.820022077396162

In [17]:
cycles = 250
number_masked = 2
result = run(text, cycles, 'frac', 0.2)
print(result)
score(result)

All of which comprises a very substantial reserve in the country at the present time . However , it will not be separated from it like other other species if its range is fully developed .


2.820022077396162

In [18]:
cycles = 100
number_masked = 1
result = run(text, cycles, 'frac', 0.2)
print(result)
score(result)

All of which comprises a very substantial reserve in the country at the present time . Although the government is not directly directly involved , in the present case , this is the the case .


2.820022077396162

In [19]:
cycles = 500
number_masked = 2
result = run(text, cycles, 'frac', 0.2)
print(result)
score(result)

All of which comprises a very substantial reserve in the country at the present time . The rest of the park is not listed on the 2008 IUCN red list , because of a landslide .


10.86345698713058

In [20]:
cycles = 1000
number_masked = 2
result = run(text, cycles, 'frac', 0.2)
print(result)
score(result)

All of which comprises a very substantial reserve in the country at the present time . All the existing protected areas are being developed , and this will will encourage further development of the resources .


2.820022077396162

In [21]:
cycles = 100
number_masked = 3
result = run(text, cycles, 'frac', 0.2)
print(result)
score(result)

All of which comprises a very substantial reserve in the country at the present time . Although the government is not directly directly involved , in the present case , this is the the case .


2.820022077396162

In [22]:
cycles = 100
number_masked = 4
result = run(text, cycles, 'frac', 0.2)
print(result)
score(result)

All of which comprises a very substantial reserve in the country at the present time . Although the government is not directly directly involved , in the present case , this is the the case .


2.820022077396162

In [23]:
cycles = 100
number_masked = 5
result = run(text, cycles, 'frac', 0.2)
print(result)
score(result)

All of which comprises a very substantial reserve in the country at the present time . Although the government is not directly directly involved , in the present case , this is the the case .


2.820022077396162

In [87]:
# def score2(original, predicted):
#     tokenize_original = gpt_tokenizer.tokenize(original)
#     tensor_original = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_original)])
    
#     tokenize_predicted = gpt_tokenizer.tokenize(predicted)
#     tensor_predicted = torch.tensor([tokenizer.convert_tokens_to_ids(tokenize_predicted)])

#     loss_fct = torch.nn.CrossEntropyLoss()
    
#     print(tensor_original.squeeze())
    
#     #loss = loss_fct(tensor_predicted.squeeze(),tensor_original.squeeze()).data 

#     return math.exp(loss)

In [38]:
fh = open('../2.ModelingGPT2andXLNet/testing_data_run/combined_data/test_gpt2_results.txt', 'r').read().splitlines()
out = open('gpt2_BERT_result.txt', 'w')

for line in fh:
    try:
        line = line.split("|")
    except:
        continue
    
    
    if len(line) == 3:
        sentence_original = line[0] + ' ' + line[1]
        original_score = score(sentence_original)

        sentence_gpt_predicted = line[0] + ' ' + line[2]
        gpt_score = score(sentence_gpt_predicted)
        
        try:
            sentence_gpt_predicted_BERTSmashed = run(sentence_gpt_predicted, 100, 'val', 2)
        except:
            print(sentence_gpt_predicted)
            continue
        
        bert_sentence = sentence_gpt_predicted_BERTSmashed.split(".")[1]
        bert_score = score(sentence_gpt_predicted_BERTSmashed)

        #print(f"{sentence_original}: {original_score}")
        #print(f"{sentence_gpt_predicted}: {gpt_score}")
        #print(f"{sentence_gpt_predicted_BERTSmashed}: {gpt_score}")

        #print("##########################")

        output = f'<RES>{line[0]}|{line[1]}|{line[2]}|{bert_sentence}|{original_score}|{gpt_score}|{bert_score}\n'
        #print(output)

        out.write(output)
    

The foreign central banks having determined that they would bring the crisis to an end , at least so far as their own countries were concerned , p. 6
I have an abiding faith in their capacity , integrity , and high purpose. "
deeply ,. .
Obviously , as long as the British Navy retains its power , no such danger exists. .
93 , leaving a surplus revenue of 30 , 340 , 577. ,
While the aggregate of taxes should not be increased , amendments might be made to the revenue laws that would , without diminishing the revenue , relieve the people from unnecessary burdens. .
26. 00
MOUNT VERNODecember 15 , 1799. 00
Whereas the territory south of the Mississippi Territory and eastward of the river Mississippi , and extending to the river Perdido , of which possession was not delivered to the United States in pursuance of the treaty concluded at Paris on the 30th April , 1803 , has at all times , as is well known , been considered and claimed by them as being within the colony of Louisiana conveyed b

In this outrage against the laws of honorable war and against the feelings sacred to humanity the British commanders can not resort to a plea of retaliation , for it is committed in the face of our example. We have been told that the British will be forced to surrender to the enemy and that they will be compelled to surrender to the British in the same way as we have been told.
They can not mitigate it by calling it a self defense against men in arms , for it embraces the most shocking butcheries of defenseless families. "I'm not saying that we can't do it, but we can't do it.
Nor can it be pretended that they are not answerable for the atrocities perpetrated , since the savages are employed with a knowledge , and even with menaces , that their fury could not be controlled. But it seems that they are not the savages, but the savages themselves.
Such is the spectacle which the deputed authorities of a nation boasting its religion and morality have not been restrained from presenting to 

Our trade , with little exception , has safely reached our ports , having been much favored in it by the course pursued by a squadron of our frigates under the command of Commodore Rodgers , and in the instance in which skill and bravery were more particularly tried with those of the enemy the American flag had an auspicious triumph. In the meantime, the British had been in the habit of making use of the English flag to their advantage, and had been obliged by the English government to give it the most favourable treatment, and to use it in the defence of their country, as well as of the whole of Europe.
The frigate Constitution , commanded by Captain Hull , after a close and short engagement completely disabled and captured a British frigate , gaining for that officer and all on board a praise which can not be too liberally bestowed , not merely for the victory actually achieved , but for that prompt and cool exertion of commanding talents which , giving to courage its highest charact

The Indian tribes not under foreign instigations remain at peace , and receive the civilizing attentions which have proved so beneficial to them. The Indians are not only able to make a living, they are also able to make a living by making their people happy.
With a view to that vigorous prosecution of the war to which our national faculties are adequate , the attention of Congress will be particularly drawn to the insufficiency of existing provisions for filling up the military establishment. It is not necessary that Congress should be compelled to make such a determination, for it will be necessary for Congress to act upon it.
Such is the happy condition of our country , arising from the facility of subsistence and the high wages for every species of occupation , that notwithstanding the augmented inducements provided at the last session , a partial success only has attended the recruiting service. But the great majority of the population has not yet been recruited.
The deficiency ha

85received on account of the loans authorized by the acts of the last session the whole sum actually obtained on loan amounts to 11the residue of which , being receivable subsequent to the 30th of September last , will , together with the current revenue , enable us to defray all the expenses of this year. The following is the statement made in the following order:
The duties on the late unexpected importations of British manufactures will render the revenue of the ensuing year more productive than could have been anticipated. The British Government is now engaged to procure for the Government of England, and to provide for the expenses of the Government, and to provide for the maintenance of its own machinery, and for the maintenance of the Government of England, the means of supplying the people of the United Kingdom with food and other necessaries.
The situation of our country , fellow citizens , is not without its difficulties , though it abounds in animating considerations , of wh

...

The grounds assumed in this annunciation amounted to an acknowledgment that the curtailment , in the extent to which it had been carried , was not necessary to the safety of the bank , and had been persisted in merely to induce Congress to grant the prayer of the bank in its memorial relative to the removal of the deposits and to give it a new charter. The prayer was made by Mr.
They were substantially a confession that all the real distresses which individuals and the country had endured for the preceding 6 or 8 months had been needlessly produced by it , with the view of affecting through the sufferings of the people the legislative action of Congress. The fact that the President of the United States had not been able to act on this point, and that he had been unable to act on the question at hand, was a very great proof that he had not acted on the question at hand.
It is subject of congratulation that Congress and the country had the virtue and firmness to bear the infliction , tha

The high handed career of this institution imposes upon the constitutional functionaries of this Government duties of the gravest and most imperative character duties which they can not avoid and from which I trust there will be no inclination on the part of any of them to shrink. I am not in favour of the establishment of a government in the form of a constitution which will be a more perfect and more democratic form of government.
My own sense of them is most clear , as is also my readiness to discharge those which may rightfully fall on me. I am not a man of any particular kind, and I have no particular interest in politics.
To continue any business relations with the Bank of the United States that may be avoided without a violation of the national faith after that institution has set at open defiance the conceded right of the Government to examine its affairs , after it has done all in its power to deride the public authority in other respects and to bring it into disrepute at home

Those institutions have already shown themselves competent to purchase and furnish domestic exchange for the convenience of trade at reasonable rates , and not a doubt is entertained that in a short period all the wants of the country in bank accommodations and exchange will be supplid as promptly and as cheaply as they have heretofore been by the Bank of the United States. In the present situation the Bank is not only in a state of disrepair, but it is in a state of distress and in a state of disrepair.
If the several States shall be induced gradually to reform their banking systems and prohibit the issue of all small notes , we shall in a few years have a currency as sound and as little liable to fluctuations as any other commercial country. The present system of banking, which is not a system of checks and balances, will be the most effective of all, and will enable the people to pay their debts, to secure their property, to secure the peace of the country, to secure their own happi

It is impossible to estimate the amount which may have been thus fraudulently obtained from the National Treasury. The amount may have been as high as $1,000,000.
I am satisfied , however , it has been such as to justify a re examination of the system and the adoption of the necessary checks in its administration. The following is a list of the checks and balances in effect at the time of the adoption of the new system:
All will agree that the services and sufferings of the remnant of our Revolutionary band should be fully compensated but while this is done , every proper precaution should be taken to prevent the admission of fabricated and fraudulent claims. We are also of the opinion that the government of the Republic of China should be held responsible for all such acts.
In the present mode of proceeding the attestations and certificates of the judicial officers of the various States from a considerable portion of the checks which are interposed against the commission of frauds. Th

We can not , therefore , watch with too much attention this arm of our defense , or cherish with too much care the means by which it can possess the necessary efficiency and extension. We are not able to see with our eyes that we are not able to see the whole world, or the whole world in a single glance.
To this end our policy has been heretofore wisely directed to the constant employment of a force sufficient to guard our commerce , and to the rapid accumulation of the materials which are necessary to repair our vessels and construct with ease such new ones as may be required in a state of war. The policy of the Government of Great Britain is that it shall be necessary to maintain a constant supply of materials for the manufacture of war, and to maintain a constant supply of materials for the maintenance and repair of the vessels which are to be used in the war, and to keep the war in readiness.
In accordance with this policy , I recommend to your consideration the erection of the add